In [119]:
import requests
import json
import pandas as pd
import numpy as np
import sqlite3

### LOADING THE DATA

In [120]:
data = pd.read_csv("/Users/ankitkothari/Documents/COMPLETED_PROJECTS/H1B_data_analysis/h1b_dump_data/optimized_h1b_data.csv")
data= data.drop(columns=['Unnamed: 0'])
data.head()

,Fiscal Year,Employer,Initial Approvals,Initial Denials,Continuing Approvals,Continuing Denials,NAICS,State,City,ZIP
0,2018,REDDY GI ASSOCIATES,0.0,0.0,0.0,1.0,99,AZ,MESA,85209.0
1,2018,ADMIRAL INSTRUMENTS LLC,1.0,0.0,0.0,0.0,33,AZ,TEMPE,85281.0
2,2018,THE BELPORT COMPANY INC,0.0,1.0,0.0,0.0,33,CA,CAMARILLO,93012.0
3,2018,CALLAWAY GOLF SALES COMPANY,1.0,0.0,0.0,0.0,33,CA,CARLSBAD,92008.0
4,2018,PAYSAFE PARTNERS LP,0.0,0.0,1.0,0.0,52,CA,IRVINE,92612.0


In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585744 entries, 0 to 585743
Data columns (total 10 columns):
Fiscal Year             585744 non-null int64
Employer                585576 non-null object
Initial Approvals       585673 non-null float64
Initial Denials         585742 non-null float64
Continuing Approvals    585626 non-null float64
Continuing Denials      585740 non-null float64
NAICS                   585744 non-null int64
State                   585581 non-null object
City                    585720 non-null object
ZIP                     585596 non-null float64
dtypes: float64(5), int64(2), object(3)
memory usage: 44.7+ MB


In [122]:
data.columns

Index(['Fiscal Year', 'Employer', 'Initial Approvals', 'Initial Denials',
       'Continuing Approvals', 'Continuing Denials', 'NAICS', 'State', 'City',
       'ZIP'],
      dtype='object')

### DROPPING COLUMNS WHICH HAVE INCOMPLETE DATA WHICH IS CRITICAL TO THE ANALYSIS

In [123]:
data= data.dropna(subset=['Fiscal Year','Employer','State','Initial Approvals',
       'Initial Denials', 'Continuing Approvals', 'Continuing Denials'])

In [124]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 585279 entries, 0 to 585743
Data columns (total 10 columns):
Fiscal Year             585279 non-null int64
Employer                585279 non-null object
Initial Approvals       585279 non-null float64
Initial Denials         585279 non-null float64
Continuing Approvals    585279 non-null float64
Continuing Denials      585279 non-null float64
NAICS                   585279 non-null int64
State                   585279 non-null object
City                    585276 non-null object
ZIP                     585254 non-null float64
dtypes: float64(5), int64(2), object(3)
memory usage: 49.1+ MB


### ADDING LATITUDE AND LONGITUDE COLUMNS FOR EACH STATE THAT WILL BE USED IN MAP PLOTS

In [125]:
us_states = data['State'].unique()
print(type(us_states))
print(us_states)

<class 'numpy.ndarray'>
['AZ' 'CA' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'ID' 'IL' 'IN' 'KS' 'KY' 'LA'
 'MA' 'MD' 'MO' 'MP' 'NC' 'ND' 'NJ' 'NY' 'OH' 'PA' 'PR' 'RI' 'SD' 'TN'
 'TX' 'VA' 'WA' 'WI' 'ME' 'AE' 'NH' 'MI' 'SC' 'WV' 'CO' 'AR' 'UT' 'OK'
 'MN' 'NE' 'VT' 'NV' 'AL' 'MS' 'IA' 'GU' 'OR' 'NM' 'AK' 'WY' 'MT' 'VI'
 '\x04' 'AP' 'FM']


In [126]:
us_states = np.delete(us_states, np.where(us_states == '\x04'))
print(us_states)

['AZ' 'CA' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'ID' 'IL' 'IN' 'KS' 'KY' 'LA'
 'MA' 'MD' 'MO' 'MP' 'NC' 'ND' 'NJ' 'NY' 'OH' 'PA' 'PR' 'RI' 'SD' 'TN'
 'TX' 'VA' 'WA' 'WI' 'ME' 'AE' 'NH' 'MI' 'SC' 'WV' 'CO' 'AR' 'UT' 'OK'
 'MN' 'NE' 'VT' 'NV' 'AL' 'MS' 'IA' 'GU' 'OR' 'NM' 'AK' 'WY' 'MT' 'VI'
 'AP' 'FM']


In [127]:
cordinates= []
points = {}
o=0
api_file = open("/Users/ankitkothari/Documents/COMPLETED_PROJECTS/H1B_data_analysis/api.txt", 'r')
api_key = api_file.read()

cordinates_us_states = pd.DataFrame()
for state in us_states:
    response = requests.get("https://maps.googleapis.com/maps/api/geocode/json?components=administrative_area:{}|country:US&key={}".format(state, api_key))
    content = response.json()
    for i in content['results']:
      cord = (i['geometry']['location'])
      for j in cord:
        cordinates_us_states.loc[o,'long'] = cord['lng']
        cordinates_us_states.loc[o,'lang'] = cord['lat']
        cordinates_us_states.loc[o,'state'] = state
    o=o+1
    cordinates.append(points)

print(cordinates_us_states)

          long       lang state
0  -111.093731  34.048928    AZ
1  -119.417932  36.778261    CA
2   -73.087749  41.603221    CT
3   -77.036871  38.907192    DC
4   -75.527670  38.910832    DE
5   -81.515754  27.664827    FL
6   -82.900075  32.165622    GA
7  -155.582782  19.896766    HI
8  -114.742041  44.068202    ID
9   -89.398528  40.633125    IL
11  -98.484246  39.011902    KS
12  -84.270018  37.839333    KY
13  -91.962333  30.984298    LA
14  -71.382437  42.407211    MA
15  -76.641271  39.045755    MD
16  -91.831833  37.964253    MO
18  -79.019300  35.759573    NC
19 -101.002012  47.551493    ND
20  -74.405661  40.058324    NJ
21  -74.005973  40.712775    NY
22  -82.907123  40.417287    OH
23  -77.194525  41.203322    PA
25  -71.477429  41.580095    RI
26  -99.901813  43.969515    SD
27  -86.580447  35.517491    TN
28  -99.901813  31.968599    TX
29  -78.656894  37.431573    VA
30 -120.740139  47.751074    WA
31  -88.787868  43.784440    WI
32  -69.445469  45.253783    ME
34  -71.

In [128]:
data = data.merge(cordinates_us_states, left_on='State', right_on='state', how='left')


In [129]:
data=data.iloc[:,0:-1]
data.head(100)

,Fiscal Year,Employer,Initial Approvals,Initial Denials,Continuing Approvals,Continuing Denials,NAICS,State,City,ZIP,long,lang
0,2018,REDDY GI ASSOCIATES,0.0,0.0,0.0,1.0,99,AZ,MESA,85209.0,-111.093731,34.048928
1,2018,ADMIRAL INSTRUMENTS LLC,1.0,0.0,0.0,0.0,33,AZ,TEMPE,85281.0,-111.093731,34.048928
2,2018,THE BELPORT COMPANY INC,0.0,1.0,0.0,0.0,33,CA,CAMARILLO,93012.0,-119.417932,36.778261
3,2018,CALLAWAY GOLF SALES COMPANY,1.0,0.0,0.0,0.0,33,CA,CARLSBAD,92008.0,-119.417932,36.778261
4,2018,PAYSAFE PARTNERS LP,0.0,0.0,1.0,0.0,52,CA,IRVINE,92612.0,-119.417932,36.778261
5,2018,STATE OF CA SECY OF STATE S OFFICE,0.0,0.0,0.0,1.0,54,CA,SACRAMENTO,95814.0,-119.417932,36.778261
6,2018,EMERALD HEALTH PHARMACEUTICALS INC,0.0,0.0,0.0,1.0,54,CA,SAN DIEGO,92121.0,-119.417932,36.778261
7,2018,GONSALVES & SANTUCCI INC DBA THE C,1.0,0.0,0.0,0.0,23,CA,SAN FRANCISCO,94111.0,-119.417932,36.778261
8,2018,A T KEARNEY,1.0,0.0,0.0,0.0,99,CA,SAN FRANCISCO,94111.0,-119.417932,36.778261
9,2018,AMERI INFO INC,1.0,0.0,0.0,0.0,54,CA,SAN JOSE,95129.0,-119.417932,36.778261


In [130]:
data.shape

(585279, 12)

### DATA CLEANING: CLEANING EMPLOYERS NAME 
    - REMOVING SPACES
    - REPLACING EMPLOYER WITH MULTIPLE NAMES DUE TO BAD DATA WITH ONE NAME

##### For example AMAZON uses variery of names to file based on there subcidiary, But they all come under Amazon so replacing them with AMAZON for analysis makes sense

In [143]:
pattern = r"\bAMAZON\b\s*.*"
amazon= data[data['Employer'].str.contains(pattern, na=False)]['Employer']
print(amazon.unique())

['AMAZON CORPORATE LLC' 'AMAZON WEB SVCS INC' 'AMAZON COM AZDC LLC'
 'AMAZON PAYMENTS INC' 'AMAZON MECHANICAL TURK INC'
 'AMAZON WEB SERVICES INC' 'AMAZON. COM SEVS INC' 'AMAZON FRESH LLC'
 'AMAZON STUDIOS LLC' 'AMAZON MEDIA GROUP LLC' 'AMAZON MEDIA GROUP'
 'AMAZON CAPITAL SERVICES INC' 'AMAZON REGISTRY SERVICES INC'
 'AMAZON RESTAURANT & BAR INC' 'AMAZON ROBOTICS LLC' 'AMAZON SERVICES LLC'
 'AMAZON FULFILLMENT SERVICES INC' 'AMAZON COM SVCS INC'
 'AMAZON DIGITAL SVCS INC' 'AMAZON COM SERVICES INC'
 'AMAZON FULFILLMENT SVCS INC' 'AMAZON COM SERVICES INC FORMERLY K'
 'AMAZON DIGITAL SERVICES INC' 'AMAZON COM INDC LLC' 'AMAZON COM KSDC LLC'
 'AMAZON.COM KYDC LLC' 'AMAZON.COM.DEDC LLC' 'AMAZON TECHNOLOGIES INC'
 'AMAZON.COM SERVICES SERVICES INC' 'AMAZON.COM SERVICES INC'
 'AMAZON COM KYDC LLC' 'AMAZON COM DEDC LLC' 'AMAZON COM AZDC INC'
 'AMAZON WEB SERVICES LLC' 'AMAZON PRODUCE NETWORK LLC'
 'AMAZON PROFESSINAL CARE' 'AMAZON SOCIETY INC' 'AMAZON.COM.NVDC INC'
 'AMAZON.COM KYDC INC' 'AMA

##### Another example DELOITTE uses variery of names to file based on there subcidiary, But they all come under DELOITTE so replacing them with DELOITTE for analysis makes sense

In [144]:
pattern = r"\bDELOITTE\b\s*.*"
amazon= data[data['Employer'].str.contains(pattern, na=False)]['Employer']
print(amazon.unique())

['DELOITTE CONSULTING OVERS' 'DELOITTE FINANCIAL ADVISORY SERVIC'
 'DELOITTE FIN ADVISORY SERVICES LLP' 'DELOITTE CONSULTING LLP'
 'DELOITTE & TOUCHE LLP' 'DELOITTE TOUCHE TOHMATSU SVCS INC'
 'DELOITTE TOUCHE TOHMATSU SVCS IN' 'DELOITTE TRANSACTIONS & BUS ANALYT'
 'DELOITTE SERVICES LP' 'DELOITTE LLP' 'DELOITTE CORPORATE FINANCE LLC'
 'DELOITTE TAX LLP' 'DELOITTE TRANSACTION & BUS ANALYTI'
 'DELOITTE FIN ADVISORY SVCS LLP' 'DELOITTE TOUCHE TOHMATSU SERVICES'
 'DELOITTE SVCS LP' 'DELOITTE LLP-DELOITTE LLP'
 'DELOITTE FINANCIAL ADVISORY SVCS' 'DELOITTE CONSULTING OVERSEAS'
 'DELOITTE CONSULTING OVERSEAS SVCS' 'DELOITTE & TOUCHE OVERSEAS SVCS LL'
 'DELOITTE CONSULTING' 'DELOITTE CONSULTING EXTENDED B'
 'DELOITTE TOUCHE OVERSEAS PROJECTS' 'DELOITTE FINANCIAL ADVISORY SRVCS'
 'DELOITTE FINANCIAL ADVISORY SVCS L' 'DELOITTE CONS OVERSEAS PROJECTS LL'
 'DELOITTE ANALYTICS LLC' 'DELOITTE & TOUCHE OVERSEAS PROJECT'
 'DELOITTE & TOUCHE USA OVERSEAS SVC' 'DELOITTE CONSULTING EXTENDED'
 'DELOITTE &

### TOP EMPLOYERS BY COUNT OF SUM OF INITIAL APPROVALS

In [151]:
top_employers_by_approvals= data.pivot_table(values=['Initial Approvals'], index='Employer', aggfunc=[np.sum] ,margins=False)
top_employers_by_approvals=top_employers_by_approvals.reset_index()

In [152]:
top_employers_by_approvals.columns

MultiIndex(levels=[['sum', 'Employer'], ['Initial Approvals', '']],
           codes=[[1, 0], [1, 0]])

In [153]:
top_employers_by_approvals= top_employers_by_approvals.sort_values(by=[('sum','Initial Approvals')], ascending=False)
top_employers_by_approvals.head()

,Employer,sum
,,Initial Approvals
119050,INTEL CORPORATION,5540.0
112945,IBM INDIA PRIVATE LIMITED,5248.0
12993,AMAZON CORPORATE LLC,4160.0
189927,PRICEWATERHOUSECOOPERS LLP,4012.0
160920,MPHASIS CORPORATION,3347.0


#### CLEANING EMPLOYER NAME BY REPLACING IT WITH ONE SHORT-CODE FOR MOST COMMON EMPLOYERS 

In [154]:
#approvals['short_code'] = approvals['Employer'].apply(lambda x: x.split(' ')[0])
top_employers_by_approvals['short_code'] = top_employers_by_approvals['Employer'].str.split().str.get(0)

In [155]:
top_employers_by_approvals.head(20)

,Employer,sum,short_code
,,Initial Approvals,
119050,INTEL CORPORATION,5540.0,INTEL
112945,IBM INDIA PRIVATE LIMITED,5248.0,IBM
12993,AMAZON CORPORATE LLC,4160.0,AMAZON
189927,PRICEWATERHOUSECOOPERS LLP,4012.0,PRICEWATERHOUSECOOPERS
160920,MPHASIS CORPORATION,3347.0,MPHASIS
83792,FACEBOOK INC,3207.0,FACEBOOK
99062,GOOGLE INC,3163.0,GOOGLE
254622,UST GLOBAL INC,2956.0,UST
176830,ORACLE AMERICA INC,2728.0,ORACLE


In [156]:
len(top_employers_by_approvals['Employer'].unique())

271494

In [162]:
top_100_employers_by_approvals= top_employers_by_approvals.head(300)
top_100_employers_by_approvals_list = top_100_employers_by_approvals['short_code'].unique()
top_100_employers_by_approvals_list = [e for e in  top_100_employers_by_approvals_list if e not in ('COMPUTER', 'CSC','RITE','TECH', 'UNIV', 'UST','THE','BANK','NEW')]
top_100_employers_by_approvals_list

['INTEL',
 'IBM',
 'AMAZON',
 'PRICEWATERHOUSECOOPERS',
 'MPHASIS',
 'FACEBOOK',
 'GOOGLE',
 'ORACLE',
 'DELOITTE',
 'MINDTREE',
 'LARSEN',
 'GOLDMAN',
 'CUMMINS',
 'KPMG',
 'APPLE',
 'QUALCOMM',
 'IGATE',
 'SYNECHRON',
 'HEXAWARE',
 'SYNTEL',
 'CAPGEMINI',
 'PATNI',
 'CISCO',
 'JPMORGAN',
 'ERNST',
 'YAHOO',
 'KPIT',
 'NVIDIA',
 'EBAY',
 'UNIVERSITY',
 'VMWARE',
 'EMC',
 'ACCENTURE',
 'BAYLOR',
 'MARLABS',
 'BIRLASOFT',
 'MCKINSEY',
 'NTT',
 'WAL-MART',
 'KFORCE',
 'BLOOMBERG',
 'BROADCOM',
 'PAYPAL',
 'ZENSAR',
 'ERICSSON',
 'POPULUS',
 'CERNER',
 'LINKEDIN',
 'EPIC',
 'TATA',
 'STANFORD',
 'INFOTECH',
 'POLARIS',
 'TEXAS',
 'PERSISTENT',
 'MAYO',
 'COLUMBIA',
 'YALE',
 'MARVELL',
 'CLEVELAND',
 'HTC',
 'HARVARD',
 'VIRTUSA',
 'NIIT',
 'MICROSOFT',
 'ADVANCED',
 'COGNIZANT',
 'NETAPP',
 'JUNIPER',
 'EMORY',
 'SCHLUMBERGER',
 'ADOBE',
 'YAHOO!',
 'GLOBALFOUNDRIES',
 'CITIGROUP',
 'HEADSTRONG',
 'WASHINGTON',
 'DELL',
 'CREDIT',
 'HITACHI',
 'MICRON',
 'EPAM',
 'ZS',
 'APPLIED',
 'CGI'

### USING PANDAS VECTORIZED FUNCTIONS CLEANING EMPLOYERS NAME

In [166]:
for employer in top_100_employers_by_approvals_list:
    pattern= r"\b{}\b\s*.*".format(employer)
    data['Employer'].str.replace(pattern, employer)

In [168]:
pattern = r"\bAMAZON\b\s*.*"
amazon= data[data['Employer'].str.contains(pattern, na=False)]['Employer']
print(amazon.unique())

['AMAZON' 'A2Z DVLP CTR INC DBA AMAZON']


In [167]:
pattern = r"\bDELOITTE\b\s*.*"
amazon= data[data['Employer'].str.contains(pattern, na=False)]['Employer']
print(amazon.unique())

['DELOITTE']


In [169]:
len(data['Employer'].unique())

268240

### WHAT ARE THE DIFFERENT NAICS CODES MEAN?

In [172]:
naics = pd.read_csv("/Users/ankitkothari/Documents/COMPLETED_PROJECTS/H1B_data_analysis/naics.csv")
naics['Sector']=naics['Sector'].astype(int)
print(naics.head())
print(data['NAICS'].head())

us_h1b_data = data.merge(naics, left_on='NAICS', right_on='Sector', how='left')
us_h1b_data.rename(columns={'Initial Approvals':'Initial_Approvals', 'Initial Denials':'Initial_Denials','Continuing Approvals':'Continuing_Approvals','Continuing Denials':'Continuing_Denials','Tax ID':'Tax_ID','Fiscal Year':'Fiscal_Year'}, inplace=True)
us_h1b_data.drop(['Sector','Name'], axis=1, inplace=True)

   Sector                                           Name
0      11     Agriculture, Forestry, Fishing and Hunting
1      21  Mining, Quarrying, and Oil and Gas Extraction
2      22                                      Utilities
3      23                                   Construction
4      31                                  Manufacturing
0    99
1    33
2    33
3    33
4    52
Name: NAICS, dtype: int64


In [173]:
us_h1b_data.head()

,Fiscal_Year,Employer,Initial_Approvals,Initial_Denials,Continuing_Approvals,Continuing_Denials,NAICS,State,City,ZIP,long,lang
0,2018,REDDY GI ASSOCIATES,0.0,0.0,0.0,1.0,99,AZ,MESA,85209.0,-111.093731,34.048928
1,2018,ADMIRAL INSTRUMENTS LLC,1.0,0.0,0.0,0.0,33,AZ,TEMPE,85281.0,-111.093731,34.048928
2,2018,THE BELPORT COMPANY INC,0.0,1.0,0.0,0.0,33,CA,CAMARILLO,93012.0,-119.417932,36.778261
3,2018,CALLAWAY GOLF SALES COMPANY,1.0,0.0,0.0,0.0,33,CA,CARLSBAD,92008.0,-119.417932,36.778261
4,2018,PAYSAFE PARTNERS LP,0.0,0.0,1.0,0.0,52,CA,IRVINE,92612.0,-119.417932,36.778261


### EXPORTING THIS DATA TO A SQLITE3 DATABASE

In [176]:
db= "us_h1b.db"
def run_query(q):
      with sqlite3.connect(db) as conn:
          return pd.read_sql(q,conn)

def run_command(c):
      with sqlite3.connect(db) as conn:
          conn.execute('PRAGMA foreign_keys = ON;')
          conn.isolation_level = None
          conn.execute(c)

def show_tables():
      q = '''
      SELECT
          name,
          type
      FROM sqlite_master
      WHERE type IN ("table","view");
      '''
      return run_query(q)

c2 = '''CREATE TABLE IF NOT EXISTS h1b (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      Fiscal_Year datetime,
      Employer text,
      Initial_Approvals INT,
      Initial_Denials int,
      Continuing_Approvals int,
      Continuing_Denials int,
      NAICS int,
      Tax_ID float,
      State  text,
      City  text,
      ZIP  int,
      long  float,
      lang  float
  ); '''

run_command(c2)

print(show_tables())

with sqlite3.connect(db) as conn:
       us_h1b_data.to_sql('h1b', conn, if_exists= 'append', index= False)



              name   type
0              h1b  table
1  sqlite_sequence  table


### Next Steps: Part 3: Using this data writing queries to analyze important H1B trends from 2009 to 2018